In [48]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [49]:
data_path = '../../data/'

In [50]:
train_df = pd.read_csv(os.path.join(data_path, 'train.csv'), index_col='id')
train_df.loc[3896, 'artist'] = 'Titian'
train_df.loc[3986, 'artist'] = 'Alfred Sisley'

vit_tta10_df = pd.read_csv(os.path.join(data_path, 'submit_vit_tta_10.csv'), index_col='id')
vit_tta7_df = pd.read_csv(os.path.join(data_path, 'submit_vit_tta_7.csv'), index_col='id')
kfold_efficient_df = pd.read_csv(os.path.join(data_path, 'submit_kfold_efficientnet.csv'), index_col='id')
kfold_caformer_df = pd.read_csv(os.path.join(data_path, 'submit_kfold_caformer.csv'), index_col='id')
soft_df = pd.read_csv(os.path.join(data_path, 'submit_soft.csv'), index_col='id')
tta5_df = pd.read_csv(os.path.join(data_path, 'submit_tta5.csv'), index_col='id')
tta5_softmax_df = pd.read_csv(os.path.join(data_path, 'submit_tta5_softmax.csv'), index_col='id')

In [51]:
from collections import Counter

picture_per_artist = Counter()
for artist in train_df['artist']:
    picture_per_artist[artist] += 1
most_common_artist = sorted(picture_per_artist, key=picture_per_artist.get, reverse=True)
most_common_artist

['Vincent van Gogh',
 'Edgar Degas',
 'Pablo Picasso',
 'Pierre-Auguste Renoir',
 'Albrecht Du rer',
 'Paul Gauguin',
 'Francisco Goya',
 'Rembrandt',
 'Titian',
 'Marc Chagall',
 'Alfred Sisley',
 'Paul Klee',
 'Rene Magritte',
 'Andy Warhol',
 'Amedeo Modigliani',
 'Henri Matisse',
 'Sandro Botticelli',
 'Mikhail Vrubel',
 'Hieronymus Bosch',
 'Leonardo da Vinci',
 'Salvador Dali',
 'Peter Paul Rubens',
 'Kazimir Malevich',
 'Pieter Bruegel',
 'Frida Kahlo',
 'Diego Velazquez',
 'Joan Miro',
 'Andrei Rublev',
 'Raphael',
 'Giotto di Bondone',
 'Gustav Klimt',
 'El Greco',
 'Jan van Eyck',
 'Camille Pissarro',
 'Edouard Manet',
 'Henri de Toulouse-Lautrec',
 'Vasiliy Kandinskiy',
 'Claude Monet',
 'Piet Mondrian',
 'Henri Rousseau',
 'Diego Rivera',
 'William Turner',
 'Edvard Munch',
 'Gustave Courbet',
 'Michelangelo',
 'Paul Cezanne',
 'Caravaggio',
 'Georges Seurat',
 'Eugene Delacroix',
 'Jackson Pollock']

In [52]:
# hard
# concaed_df = pd.concat([kfold_df, tta10_df, tta7_df, tta5_df, ensemble_df], axis=1)
concaed_df = pd.concat([vit_tta10_df, kfold_efficient_df, kfold_caformer_df, tta5_df, tta5_softmax_df, soft_df], axis=1)
concaed_df.columnus = ['tta10', 'kfold-efficient', 'kfold-caformer', 'tta5', 'tta5_softmax', 'soft_df']
concaed_df.head()

C:\Users\dukim\AppData\Local\Temp\ipykernel_16880\3236891715.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  concaed_df.columnus = ['tta10', 'kfold-efficient', 'kfold-caformer', 'tta5', 'tta5_softmax', 'soft_df']


,artist,artist,artist,artist,artist,artist
id,,,,,,
TEST_00000,Edgar Degas,Edgar Degas,Edgar Degas,Edgar Degas,Edgar Degas,Edgar Degas
TEST_00001,Amedeo Modigliani,Amedeo Modigliani,Amedeo Modigliani,Amedeo Modigliani,Amedeo Modigliani,Amedeo Modigliani
TEST_00002,Caravaggio,Caravaggio,Caravaggio,Caravaggio,Caravaggio,Caravaggio
TEST_00003,Albrecht Du rer,Albrecht Du rer,Albrecht Du rer,Albrecht Du rer,Albrecht Du rer,Albrecht Du rer
TEST_00004,Pablo Picasso,Edgar Degas,Pablo Picasso,Pablo Picasso,Pablo Picasso,Pablo Picasso


In [53]:
preds_mode = concaed_df.mode(axis=1)
preds_mode

,0,1,2
id,,,
TEST_00000,Edgar Degas,NaN,NaN
TEST_00001,Amedeo Modigliani,NaN,NaN
TEST_00002,Caravaggio,NaN,NaN
TEST_00003,Albrecht Du rer,NaN,NaN
TEST_00004,Pablo Picasso,NaN,NaN
...,...,...,...
TEST_12665,Edgar Degas,NaN,NaN
TEST_12666,Diego Velazquez,NaN,NaN
TEST_12667,Amedeo Modigliani,NaN,NaN


In [54]:
preds = preds_mode[0].copy()
preds

id
TEST_00000          Edgar Degas
TEST_00001    Amedeo Modigliani
TEST_00002           Caravaggio
TEST_00003      Albrecht Du rer
TEST_00004        Pablo Picasso
                    ...        
TEST_12665          Edgar Degas
TEST_12666      Diego Velazquez
TEST_12667    Amedeo Modigliani
TEST_12668               Titian
TEST_12669     Vincent van Gogh
Name: 0, Length: 12670, dtype: object

In [55]:
preds_mode[~preds_mode[1].isnull()]

,0,1,2
id,,,
TEST_00051,Kazimir Malevich,Vasiliy Kandinskiy,NaN
TEST_00262,Gustav Klimt,Leonardo da Vinci,NaN
TEST_00263,Edgar Degas,Edouard Manet,NaN
TEST_00291,Pablo Picasso,Vincent van Gogh,NaN
TEST_00321,Pablo Picasso,Vincent van Gogh,NaN
...,...,...,...
TEST_12027,Georges Seurat,Vincent van Gogh,NaN
TEST_12034,Edgar Degas,Pablo Picasso,NaN
TEST_12111,Sandro Botticelli,Titian,NaN


In [56]:
def get_most_common_artist(artists, target):
    for artist in artists:
        if artist in target:
            return artist
preds_mode[~preds_mode[1].isnull()].apply(lambda x: get_most_common_artist(most_common_artist, x.values), axis=1)

id
TEST_00051     Kazimir Malevich
TEST_00262    Leonardo da Vinci
TEST_00263          Edgar Degas
TEST_00291     Vincent van Gogh
TEST_00321     Vincent van Gogh
                    ...        
TEST_12027     Vincent van Gogh
TEST_12034          Edgar Degas
TEST_12111               Titian
TEST_12392               Titian
TEST_12431        Pablo Picasso
Length: 115, dtype: object

In [57]:
# preds.loc[preds_mode[~preds_mode[1].isnull()].index] = tta10_df.loc[preds_mode[~preds_mode[1].isnull()].index].values.reshape(-1)
# preds.loc[preds_mode[~preds_mode[1].isnull()].index] = preds_mode[~preds_mode[1].isnull()].apply(lambda x: get_most_common_artist(most_common_artist, x.values), axis=1)
# preds

In [58]:
preds.loc[preds_mode[~preds_mode[1].isnull()].index] = soft_df.loc[preds_mode[~preds_mode[1].isnull()].index].values.reshape(-1)

In [59]:
submit = pd.read_csv(os.path.join(data_path, './sample_submission.csv'))

In [60]:
submit['artist'] = preds.values

In [61]:
submit.head()

,id,artist
0,TEST_00000,Edgar Degas
1,TEST_00001,Amedeo Modigliani
2,TEST_00002,Caravaggio
3,TEST_00003,Albrecht Du rer
4,TEST_00004,Pablo Picasso


In [62]:
submit.to_csv(os.path.join(data_path, f"./submit_hard.csv"), index=False)